<a href="https://colab.research.google.com/github/ActiveStacks/NLP/blob/main/MiniThesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

# CLASSIFIERS
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix


In [ ]:
# path = ('.\LVDAndro_APKs_Combined_Processed')
# df = pd.read_csv(path)
# df=pd.read_excel(r'TT.xlsx',skiprows=1)
# path = ('.\LVDAndro_All_MobSF_Processed.csv')

path = ('/content/drive/MyDrive/LVDAndro_SourceFiles_Qark_Processed.csv')
# path = ('/content/drive/MyDrive/XSS_dataset.csv')
df = pd.read_csv(path)

In [ ]:
df

,index,category,Severity,Description,Code,CWE_ID,CWE_Desc,Vulnerability_status,processed_code
0,0,NaN,NaN,NaN,package in.ac.dducollegedu.shell;,NaN,NaN,0,package in.ac.dducollegedu.shell;
1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,2,NaN,NaN,NaN,import android.content.Context;,NaN,NaN,0,import android.content.Context;
3,3,NaN,NaN,NaN,import androidx.test.platform.app.Instrumentat...,NaN,NaN,0,import androidx.test.platform.app.Instrumentat...
4,4,NaN,NaN,NaN,import androidx.test.ext.junit.runners.Android...,NaN,NaN,0,import androidx.test.ext.junit.runners.Android...
...,...,...,...,...,...,...,...,...,...
595057,618370,NaN,NaN,NaN,browserWebview.getSettings().setJavaScriptEnab...,NaN,NaN,0,browserWebview.getSettings().setJavaScriptEnab...
595058,618371,file,WARNING,Logging found,"Log.w(TAG, ""Native error "" + error + "": "" + me...",CWE-532,Insertion of Sensitive Information into Log File,1,"Log.w(TAG, ""user_str"" + error + ""user_str"" + m..."
595059,618372,file,WARNING,Logging found,"Log.i(TAG, ""MTU="" + mtu);",CWE-532,Insertion of Sensitive Information into Log File,1,"Log.i(TAG, ""user_str"" + mtu);"
595060,618373,file,WARNING,Logging found,"Log.e(log_tag, ""Failed to copy asset file: "" +...",CWE-532,Insertion of Sensitive Information into Log File,1,"Log.e(log_tag, ""user_str"" + filename, e);"


In [ ]:

df.drop("index", axis = 1, inplace=True)
df.drop("category", axis = 1, inplace=True)
df.drop("Severity", axis = 1, inplace=True)
df.drop("Description", axis = 1, inplace=True)

df.drop("Code", axis = 1, inplace=True)
df.drop("CWE_ID", axis = 1, inplace=True)
df.drop("CWE_Desc", axis = 1, inplace=True)


In [ ]:
df

,Vulnerability_status,processed_code
0,0,package in.ac.dducollegedu.shell;
1,0,NaN
2,0,import android.content.Context;
3,0,import androidx.test.platform.app.Instrumentat...
4,0,import androidx.test.ext.junit.runners.Android...
...,...,...
595057,0,browserWebview.getSettings().setJavaScriptEnab...
595058,1,"Log.w(TAG, ""user_str"" + error + ""user_str"" + m..."
595059,1,"Log.i(TAG, ""user_str"" + mtu);"
595060,1,"Log.e(log_tag, ""user_str"" + filename, e);"


In [ ]:
df.describe()

,Vulnerability_status
count,595062.000000
mean,0.009654
std,0.097782
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
df['Vulnerability_status']

0         0
1         0
2         0
3         0
4         0
         ..
595057    0
595058    1
595059    1
595060    1
595061    1
Name: Vulnerability_status, Length: 595062, dtype: int64

In [ ]:
df['processed_code'] = df['processed_code'].apply(str)

In [ ]:
def code_tok(text):
    return " ".join(word for word in word_tokenize(text))

In [ ]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['processed_code'] = df['processed_code'].apply(lambda x: code_tok(x))

In [ ]:
df['processed_code']

0                         package in.ac.dducollegedu.shell;
1                                                       nan
2                           import android.content.Context;
3         import androidx.test.platform.app.Instrumentat...
4         import androidx.test.ext.junit.runners.Android...
                                ...                        
595057    browserWebview.getSettings().setJavaScriptEnab...
595058    Log.w(TAG, "user_str" + error + "user_str" + m...
595059                        Log.i(TAG, "user_str" + mtu);
595060            Log.e(log_tag, "user_str" + filename, e);
595061                     Log.w(TAG, "user_str" + reason);
Name: processed_code, Length: 595062, dtype: object

In [ ]:
X = df['processed_code']
y = df['Vulnerability_status']

In [ ]:
vectorCount = CountVectorizer()


In [ ]:
X_vect = vectorCount.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size = 0.2, random_state = 1)

In [ ]:
classifier_tree = DecisionTreeClassifier(random_state=42)

In [ ]:
model_CART = classifier_tree.fit(X_train, y_train)

In [ ]:
y_predict = model_CART.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(f"Accuracy of Decision on test set : {accuracy_score(y_test, y_predict)}")
print(f"Precion of Decision Tree on test set : {precision_score(y_test, y_predict)}")
print(f"Recall of Decision Tree on test set : {recall_score(y_test, y_predict)}")

Accuracy of Decision on test set : 0.9836068328669977
Precion of Decision Tree on test set : 0.09287257019438445
Recall of Decision Tree on test set : 0.07184628237259816


In [ ]:
print(accuracy)
print(precision)

0.9836068328669977
0.09287257019438445


In [ ]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    117816
           1       0.09      0.07      0.08      1197

    accuracy                           0.98    119013
   macro avg       0.54      0.53      0.54    119013
weighted avg       0.98      0.98      0.98    119013

